In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.interpolate import interp1d
import scipy.optimize as so
import tmm_core as tmm
import lmfit
from matplotlib import pyplot as plt

In [2]:
class Material:
    def __init__(self, path, name, wavelengths):
        f = pd.read_csv(path)
        wv_raw = np.array(f.wv)
        nc = np.array(f.n+f.k*1j)  # complex refractive index
        self.name = name
        self.f = interp1d(wv_raw, nc, kind='cubic')
        self.min_wv = min(wv_raw)
        self.max_wv = max(wv_raw)
        self.df = pd.Series(data=nc, index=wv_raw, name=name)
        # self.df = self.df.reindex(wavelengths)
        # self.df = self.df.interpolate('spline', order=3)
        self.interp(wavelengths)

    def interp(self, wavelengths):
        wavelengths = wavelengths[np.nonzero(np.logical_and(wavelengths > self.min_wv, wavelengths < self.max_wv))]
        nc = self.f(wavelengths)
        self.df = pd.DataFrame(nc,wavelengths, [self.name])


In [34]:
class Model:
    def __init__(self):
        self.wavelength = np.arange(300, 1700, 1)
        self.increment = .2
        self.index_array = np.array([])
        self.materials = []
        self.mat_df = pd.DataFrame([], self.wavelength)
        self.data = {}
        self.thicknesses = [sp.inf, 0, sp.inf]
        self.layers = []

        self.add_material("TCO", './Materials/Semiconductor/TCO.csv')
        self.add_material("CdSe", './Materials/Semiconductor/CdSe.csv')
        self.add_material("CdTe", './Materials/Semiconductor/CdTe.csv')
        self.add_material("Air", './Materials/Semiconductor/Air.csv')
        self.add_material("BK7", './Materials/Dielectric/BK7.csv')
        self.add_material("SLG", './Materials/Dielectric/SLG.csv')
        self.add_material("SS TCO", './Materials/Dielectric/Sunnyside TCO.csv')
        self.add_material("SiO2", './Materials/Dielectric/SiO2-Jaw.csv')
        
    def add_material(self, film, path):
        if film not in self.mat_df:
            mat = Material(path, film, self.wavelength)
            self.materials.append(mat)
            self.mat_df = self.mat_df.join(mat.df)

    def set_wavelength(self, low, high, interval):
        self.wavelength = np.arange(low, high, interval)
        df = self.mat_df.reindex(self.wavelength)
        df = df.interpolate('spline', order=3)
        self.mat_df = df

    def better_bruggeman(self, n1, n2, percent_included):
        p = n1/n2
        b = .25*((3*percent_included-1)*(1/p-p)+p)
        z = b + (b**2 + .5)**0.5
        e = z*n1*n2
        return {"e": e, "n": e**0.5, 'conc': percent_included, "n1": n1, 'n2': n2}

    def brug_transform(self, df, layer, incl, percent):
        p = df[layer]/incl
        b = .25*((3*percent-1)*(1/p-p)+p)
        z = b + (b**2 + .5)**0.5
        e = z*df[layer]*incl
        n = e**.5
        df[layer] = n
    
    def run(self, wavelengths, void_percent):
        mat = self.mat_df.ix[wavelengths]
        mat = mat[self.layers]
        self.brug_transform(mat, self.layers[1], mat['Air'], void_percent)
        self.index_array = np.array(mat)
        theta0 = 45*sp.pi/180
        self.data = tmm.unpolarized_RT(self.index_array, self.thicknesses, theta0, wavelengths)
        
    def normalized(a, axis=-1, order=2):
        l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
        l2[l2==0] = 1
        return a / np.expand_dims(l2, axis)
    
    def get_R(self, wavelengths, thickness, void_percent):
        self.thicknesses[1] = thickness
        self.run(wavelengths, void_percent) 
        R = self.data['R']
        R -= min(R)
        R /= max(R)
        return R
        
    def RMSE(self, thickness, data):
        df = pd.DataFrame(data)
        self.get_R(thickness)
        model = pd.DataFrame(self.data['R'], index=self.wavelength)
        df = df.join(model, how='inner')
        n = len(df.index)
        return (sum((data-model)**2)/n)**0.5
    
    def norm(self, wavelength):
        df = self.df.ix[wavelength]
        df = df - df.min()
        df = df / df.max()
        return df
    
    def fit(self, wv, data):
        mod = lmfit.Model(self.get_R, ['wavelengths'], ['thickness','void_percent'])
        mod.set_param_hint('thickness', value = 130, min=50, max=250)
        mod.set_param_hint('void_percent', value = .15, min=.05, max=.5)

        R = data.series.ix[wv]
        result = mod.fit(R, wavelengths=wv)

        RMSE = (sp.sum(result.residual**2)/(result.residual.size-2))**0.5
        bf_values = result.best_values
        bf_str = 'thk: ' + str(round(bf_values['thickness'])) +", Void %: " + str(round(bf_values['void_percent']*100, 2))
        txt_spot = wv.min()-100 + (wv.max()-wv.min()) / 2

        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.text(txt_spot, .9, "RMSE: "+str(round(RMSE, 3)))
        ax.text(txt_spot, .85, bf_str)
        result.plot_fit(data_kws ={'marker':'+'})

        plt.show()
        print(result.fit_report())
    

In [62]:
class Data:
    def __init__(self, path, name, wavelengths, c_type):
        f = pd.read_csv(path)
        wv_raw = np.array(f.wv)
        R = np.array(f.R)
        self.name = name
        self.f = interp1d(wv_raw, R, kind='cubic')
        self.min_wv = min(wv_raw)
        self.max_wv = max(wv_raw)
        self.raw_series = pd.Series(data=R, index=wv_raw, name=name)
        self.interp(wavelengths)
        self.series = self.norm(wavelengths)
        
    def interp(self, wavelengths):
        wavelengths = wavelengths[np.nonzero(np.logical_and(wavelengths > self.min_wv, wavelengths < self.max_wv))]
        data = self.f(wavelengths)
        self.raw_series = pd.Series(data=data, index=wavelengths, name=self.name)
    
    def norm(self, wavelength, data_filter="Savitzky-Golay"):
        s = self.raw_series.ix[wavelength]
        s = s.dropna()
        s = s - s.min()
        s = s / s.max()
        if data_filter == "Savitzky-Golay":
            arr = self.savitzky_golay(y=np.array(s), window_size=11, order=3, deriv=0, rate=1)
            s = pd.Series(arr, s.index, name=self.raw_series.name+" processed")
        return s
    
    def savitzky_golay(self, y, window_size, order, deriv=0, rate=1):
        r"""Smooth (and optionally differentiate) data with a Savitzky-Golay filter.
        The Savitzky-Golay filter removes high frequency noise from data.
        It has the advantage of preserving the original shape and
        features of the signal better than other types of filtering
        approaches, such as moving averages techniques.
        Parameters
        ----------
        y : array_like, shape (N,)
            the values of the time history of the signal.
        window_size : int
            the length of the window. Must be an odd integer number.
        order : int
            the order of the polynomial used in the filtering.
            Must be less then `window_size` - 1.
        deriv: int
            the order of the derivative to compute (default = 0 means only smoothing)
        rate: int
            don't know
        Returns
        -------
        ys : ndarray, shape (N)
            the smoothed signal (or it's n-th derivative).
        Notes
        -----
        The Savitzky-Golay is a type of low-pass filter, particularly
        suited for smoothing noisy data. The main idea behind this
        approach is to make for each point a least-square fit with a
        polynomial of high order over a odd-sized window centered at
        the point.
        Examples
        --------
        t = np.linspace(-4, 4, 500)
        y = np.exp( -t**2 ) + np.random.normal(0, 0.05, t.shape)
        ysg = savitzky_golay(y, window_size=31, order=4)
        import matplotlib.pyplot as plt
        plt.plot(t, y, label='Noisy signal')
        plt.plot(t, np.exp(-t**2), 'k', lw=1.5, label='Original signal')
        plt.plot(t, ysg, 'r', label='Filtered signal')
        plt.legend()
        plt.show()
        References
        ----------
        .. [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of
           Data by Simplified Least Squares Procedures. Analytical
           Chemistry, 1964, 36 (8), pp 1627-1639.
        .. [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing
           W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery
           Cambridge University Press ISBN-13: 9780521880688
        """
        import numpy as np
        from math import factorial

        try:
            window_size = np.abs(np.int(window_size))
            order = np.abs(np.int(order))
        except (ValueError, msg):
            raise ValueError("window_size and order have to be of type int")
        if window_size % 2 != 1 or window_size < 1:
            raise TypeError("window_size size must be a positive odd number")
        if window_size < order + 2:
            raise TypeError("window_size is too small for the polynomials order")
        order_range = range(order+1)
        half_window = (window_size -1) // 2
        # precompute coefficients
        b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
        m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
        # pad the signal at the extremes with
        # values taken from the signal itself
        firstvals = y[0] - np.abs(y[1:half_window+1][::-1] - y[0])
        lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
        y = np.concatenate((firstvals, y, lastvals))
        return np.convolve(m[::-1], y, mode='valid')

In [35]:
model = Model()

In [63]:
data = Data('C:/WORK!!!!/VF088 - ARC Thickness/EKE-HC d.csv', 'Data', np.arange(300,1700), 'R')

In [65]:
wv = np.arange(400, 900)
s1 = data.norm(wv)
s2 = data.norm(wv, None)

s1.name = 'sav-golay'
s2.name = 'Just Norm'
x = pd.concat([data.raw_series, data.series], axis=1).dropna()

plt.plot(x.index, x['Data processed'])
plt.show()

In [38]:
model.layers = ['Air', 'BK7', 'SLG']
wv = np.arange(400, 900)
data.series = data.norm(wv)
model.fit(wv, data)

 - Adding parameter for hint "thickness"
 - Adding parameter for hint "void_percent"
 - Adding parameter "thickness"
 - Adding parameter "void_percent"
[[Model]]
    Model(get_R)
[[Fit Statistics]]
    # function evals   = 49
    # data points      = 500
    # variables        = 2
    chi-square         = 0.906
    reduced chi-square = 0.002
[[Variables]]
    thickness:      113.301370 +/- 1.757884 (1.55%) (init= 130)
    void_percent:   0.05000000 +/- 7.04e-06 (0.01%) (init= 0.15)
[[Correlations]] (unreported correlations are <  0.100)
    C(thickness, void_percent)   = -0.998 



In [105]:
extrema_criterion = .00005
ex = data.df.diff().abs() < extrema_criterion
ex[ex == True]

558    True
587    True
599    True
616    True
620    True
624    True
626    True
628    True
661    True
776    True
801    True
894    True
Name: Data, dtype: bool

NameError: name 'xenon' is not defined

In [ ]:
def bruggeman(n1, n2, percent_included):
    e1 = n1**2
    e2 = n2**2
    b = ((3*percent_included-1)*(e2-e1)+e1)/4
    e = b + (b + e1*e2/2)**0.5
    e_neg = b - (b + e1*e2/2)**0.5
    d = {"e": e, "e_neg": e_neg, "n": e**.5, "n_neg": e_neg**.5, 'conc':percent_included, "n1":n1, 'n2':n2}
    return(d)

def better_bruggeman(n1, n2, percent_included):
    p = n1/n2
    b = .25*((3*percent_included-1)*(1/p-p)+p)
    z = b + (b**2 + .5)**0.5
    e = z*n1*n2
    return({"e":e, "n":e**0.5, 'conc':percent_included, "n1":n1, 'n2':n2})

def brug_transform(df, layer, incl, percent):
    p = df.ix(layer)/incl
    b = .25*((3*percent-1)*(1/p-p)+p)
    z = b + (b**2 + .5)**0.5
    e = z*df[layer]*incl
    n = e**.5
    df[layer] = n

In [50]:
layers = ['Air', 'BK7', 'SLG']
wv = np.arange(400, 900)
mod = lmfit.Model(model.get_R, ['wavelengths'], ['thickness','void_percent', 'layers'])
mod.set_param_hint('thickness', value = 130, min=50, max=250)
mod.set_param_hint('void_percent', value = .15, min=.05, max=.5)
mod.set_param_hint('layers', value=layers, vary=False)

R = normal.norm(wv)
result = mod.fit(R, wavelengths=wv)

 - Adding parameter for hint "thickness"
 - Adding parameter for hint "void_percent"
 - Adding parameter for hint "layers"
 - Adding parameter "thickness"
 - Adding parameter "void_percent"
 - Adding parameter "layers"


IndexError: list index out of range

In [ ]:
normal.df